# 제9장 SHAP테이블 데이터 활용

## 설명을 위한 준비
### 환경 구축

In [ ]:
from pprint import pprint
import pandas as pd
import sklearn
import lightgbm as lgb
import shap
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
plt.style.use("ggplot")
sns.set(font="meiryo")

### 데이터 세트 준비

In [ ]:
# Prediction task is to determine whether a person makes over 50K a year.
X,y = shap.datasets.adult()
# 설명 변수
display(X.head())
# 목적 변수
print(y[:10])
# 데이터 사이즈
print(X.shape)
# 각 열의 데이터 형
print(X.dtypes)
# 각 열의 결손수 : 결손은 없으므로 이번에는 전처리를 필요로 하지 않음
print(X.isna().sum())

- "Workclass"나"Marital Status"등의 데이터형을 명시적으로 카테고리 형으로 지정해둡니다.

In [ ]:
for c in X.columns:
    if X[c].dtype==np.int8:
        # 데이터 형을 변경
        X[c] = X[c].astype("category")
print(X.dtypes)

- 설명 변수 및 목적 변수를 학습용과 테스트용으로 각각 무작위로 분할해 둡니다.

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=0)

    ### 모델 준비

### LightGBM모델 학습과 평가

- LGBMClassifier
- 예측값 (확률)의 평가 지표 : AUC-ROC 점수및 혼돈 행렬

In [ ]:
# 모델 학습: 
model = lgb.LGBMClassifier()
model.fit(X_train.values, y_train)
# 예측값 산출 
# predict_prob의 반환값은 (0의 확률, 1의 확률의 2개 열이므로 1의 확률만 추출해둠
y_pred_prob = model.predict_proba(X_test)[:, 1] 
y_pred = model.predict(X_test)
# 예측값의 평가 
val = sklearn.metrics.roc_auc_score(y_test, y_pred_prob)
print(f"auc-rocスコア: {val}")
display(pd.crosstab(y_test, y_pred))

- 튜닝을 하지 않으므로 모델 파라미터는 초기값 상태

In [ ]:
pprint(model.get_params())

## SHAP 적용
1. SHAP값의 산출과 이해
2. SHAP값의 가시화
3. SHAP값의 분석

### SHAP값의 산출과 이해
- TreeSHAP\[tree\]

In [ ]:
# 설명기 준비～SHAP값 산출
exp = shap.TreeExplainer(model)
sv_test = exp.shap_values(X_test)
sv_test = sv_test[1]
# 원래의 데이터 세트와 같은 사이즈의 행렬이 구해짐
print(X_test.shape, sv_test.shape)
print(sv_test[0])

In [ ]:
print(exp.expected_value[1] + sv_test[0].sum(), y_pred_prob[0])

- 계산 결과 : 양측이 일치하지 않음
⇒모든 예측 대상에 대해서 (SHAP값의 합계, 예측치)를 계산

In [ ]:
# SHAP값의 합계 + 평균값
sv_sum = sv_test.sum(axis=1) + exp.expected_value[1]
plt.scatter(sv_sum, y_pred_prob, s=1)
plt.xlabel("SHAP값의 합계")
plt.ylabel("모델의 예측확률")


$$y = \frac{1}{1 + \mathrm{exp}(-x)}$$

로 표현되는 시그모이드 함수로 나타남
⇒SHAP값에서 원래 모델의 확률 예측 값을 복원한 경우 SHAP의 총합에 시그모이드 함수를 적용

In [ ]:
prob_shap = (1/(1+np.exp(-sv_sum))) # SHAP값에서 원복한 모델의 예측 확률
assert np.all(np.isclose(prob_shap, y_pred_prob))

#### SHAP값에 대해 정리
```
모델 -> 예측값(실수값 오즈) -> (시그모이드 함수) -> 예측값(0-1확률)
           v (평균값을 공제함)
        정규화 한 오즈      -> (영향도 계산) ->  SHAP값
```

### 컬럼: LightGBM의 SHAP연계기능

- LightGBM에 의한 SHAP값 계산

In [ ]:
pred = model.predict_proba(X_test)[:,1] # 원래의 예측확률
res = model.predict(X_test, pred_contrib=True) # SHAP값 행렬 복원
# TODO: SHAP라이브러리에서 구한 값과의 비교
pred2 = (1/(1+np.exp(-(res[:,:-1].sum(axis=1) + res[:,-1])))) # 예측확률을 복원
assert np.all(np.isclose(pred,pred2))

## SHAP값의 가시화에 의한 모델 이해

In [ ]:
# shap.initjs() # javascript를 사용하기 위해 필요. javescript환경이 아닌 경우는 matplotlib인수를 True로 부여함
shap.force_plot(exp.expected_value[1], sv_test[0], X_test.iloc[0], matplotlib=True)

#### force_plot  
Relationship, Hours per week이라는 특징량이 크고 그것은 마이너스 방향으로 영향을 미침

In [ ]:
# decision_plot: 특징량의 수가 많은 경우
shap.decision_plot(exp.expected_value[1], sv_test[0], X_test.iloc[0])

#### decision_plot  
⇒반수 정도의 특징량은 거의 영향을 주지 않음

### 데이터 세트 전체에 대한 특징량의 반영 방법

In [ ]:
shap.summary_plot(sv_test, X_test)

#### summary_plot
* 대략적으로 연령이 낮으면 마이너스 방향으로 높으면 플러스 방향으로 영향을 미치고 있음
* 연령이 높은 경우의 플러스 방향의 영향보다 연령이 작은 경우의 마이너스 방향의 영향이 더 큼
* 대부분의 경우 영향의 절대값은 작으며 크게 기여하는 경우는 일부 경우에 한함 

#### 특징량의 중요도만 알고 싶은 경우

In [ ]:
shap.summary_plot(sv_test, X_test, plot_type="bar")

### SHAP값과 특징량의 상관 관계를 가시화

In [ ]:
shap.dependence_plot("Relationship", sv_test, X_test)

#### dependence_plot
* SHAP값으로는 값이 4와 5의 경우 플러스, 그 이외에는 마이너스 방향으로 기여하는 경향이 있음
* Relationship의 값이 4,5이고 Capital Gain의 값이 큰 경우 마이너스 방향으로 상호 작용이 존재함

In [ ]:
shap.dependence_plot("Age", sv_test, X_test)

#### dependence_plot
* 연령이 낮은 경우에는 전체적으로 예측값이 낮아지지만 예외적으로 그렇게 낮지 않게 영향을 미치지 않는 경우가 있고 그렇나 경우에는 교육 수준이 높지 않다.
* 연령이 일정 이상인 경우에는 연령이 대체적으로 동일하게 플러스 방향으로 영향을 미치지만 교육 수준이 높은 경우에는 보다 커지고 낮은 경우에는 작아짐 (상호작용이 있음)
* 연령이 60세 이상의 경우에 세로 방향을 분산되며 특히 예측값이 낮은 경우가 있으나 그 원인은 이 그림만으로는 알 수 없음 (Education-num 이외의 특징량과의 상호작용이 있다고 생각됨)

### コラム: 변수 사이의 상호 작용에 의한 상세 관찰 : SHAP Interaction Values
#### Interaction Values
* 2개의 특징량의 상호작용만을 빼내어 산출함
* 1개의 특징량에 대해 전체의 효과에서 다른 변수와의 상호작용을 제외하고 독자적인 영향만을 산출함

In [ ]:
iv = exp.shap_interaction_values(X_test) # interaction value의 산출
print(iv.shape) # ( 데이터수 , 특징량수 , 특징량수 ) 의 사이즈
assert np.all(np.isclose(sv_test[:,0], iv[:,0,:].sum(axis=1))) # 특징량에 대해 산출하면 SHAP 값과 일치함

In [ ]:
# Age" 의 영향 (SHAP 값 ) 에서 그 외의 특징량의 상호작용을 제외한 그래프
# 세로 방향의 분산도가 작아지고 있음
shap.dependence_plot(
    ("Age", "Age"),
    iv, X_test,
    display_features=X_test
)

In [ ]:
# "Age"와"Education-Num"의 상호작용 그래프
#  dependence_plot와 동일한 경향의 상호 작용을 확인 가능함
shap.dependence_plot(
    ("Age", "Education-Num"),
    iv, X_test,
    display_features=X_test
)